<a href="https://colab.research.google.com/github/oyinig/African-AI-Foundation/blob/main/Generative_AI_Demos_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s eta 0:00:00


##Music Generation

In [ ]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import scipy

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

inputs = processor(
    text=["80s pop track with bassy drums and synth", "90s rock song with loud guitars and heavy drums"],
    padding=True,
    return_tensors="pt",
)
audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)

#save audio from array to a wav file on disk
sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=audio_values[0, 0].numpy())
audio_values

tensor([[[ 0.0995,  0.0953,  0.1111,  ...,  0.0229,  0.0013,  0.0261]],

        [[ 0.0652,  0.0636,  0.1194,  ..., -0.0324, -0.0311, -0.0288]]])

In [ ]:
from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(audio_values[0].numpy(), rate=sampling_rate)

##Text to speech

Showing an example of how to load bark from the transformer library

Change voice_preset to different speakers to have different output

Refer to https://suno-ai.notion.site/8b8e8749ed514b0cbf3f699013548683?v=bc67cff786b04b50b3ceb756fd05f68c for more information

In [19]:
from transformers import AutoProcessor, BarkModel
import torch
from IPython.display import Audio

device = "cuda" if torch.cuda.is_available() else "cpu"

# load in fp16
processor = AutoProcessor.from_pretrained("suno/bark-small", torch_dtype=torch.float16)
model = BarkModel.from_pretrained("suno/bark-small", torch_dtype=torch.float16).to(device)


voice_preset = "v2/en_speaker_6"

inputs = processor("Hello uh ... [clears throat], my dog is cute [laughter]", voice_preset=voice_preset).to(device)

audio_array = model.generate(**inputs)
audio_array = audio_array.cpu().numpy().squeeze()



sampling_rate = model.generation_config.sample_rate
Audio(audio_array, rate=sampling_rate)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


##Facebook MMS Model For Audio Generation

In [20]:
import torch
from transformers import VitsTokenizer, VitsModel, set_seed

tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-eng")
model = VitsModel.from_pretrained("facebook/mms-tts-eng")

inputs = tokenizer(text="Hello - my dog is really really cute", return_tensors="pt")

set_seed(120)  # make deterministic

with torch.no_grad():
   outputs = model(**inputs)

waveform = outputs.waveform[0]
sampling_rate = model.config.sampling_rate
Audio(waveform.numpy(), rate=sampling_rate)

#waveform

##Pop to Piano

In [1]:
!pip install pretty-midi==0.2.9
!pip install datasets
!pip install essentia==2.1b6.dev1034
!pip install scipy
!pip install librosa
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 5.6 MB/s eta 0:00:00
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591932 sha256=cea7f1f0c88ded0bd869f4ca0eb39d316ca0b24223fae71c25b3db6c3a54055f
  Stored in directory: /root/.cache/pip/wheels/f5/05/68/55f44833c1801964b3ef0881e1798952bf53352a89d3a5932c
Successfully built pretty-midi
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━

#Speech to text
Convert speech to text using OPEN AI whisper model

In [3]:
import torch
from transformers import AutoProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

processor = AutoProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

inputs = processor(ds[1]["audio"]["array"], return_tensors="pt")
input_features = inputs.input_features

generated_ids = model.generate(inputs=input_features)

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
transcription

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


" Nor is Mr. Quilter's manner less interesting than his matter."